# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
             You are an SQL assistant. Generate SQLite-compatible SQL queries based on user requests.
             Here is the database schema:
-- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-5678'),
(3, 'Charlie', 'Sales', '2020-01-15', 'charlie@example.com', '555-9876'),
(4, 'Diana', 'Marketing', '2019-05-30', 'diana@example.com', '555-2468');

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,              -- Employee ID from the employees table
    year INT,                -- Year of salary information
    base FLOAT,              -- Base salary of the employee
    bonus FLOAT,             -- Bonus given to the employee
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links salary to employee
);

INSERT INTO salary VALUES
(1, 2022, 50000, 1000),
(2, 2022, 70000, 3000),
(3, 2022, 60000, 1500),
(4, 2022, 65000, 1200);

-- Table: studies
CREATE TABLE studies (
    ID INT PRIMARY KEY,      -- Unique ID for each study record
    ID_usr INT,              -- Employee ID linking to the employees table
    degree TEXT,             -- Degree earned by the employee
    institution TEXT,        -- Institution where the degree was obtained
    graduation_year INT,     -- Year the employee graduated
    major TEXT,              -- Major or field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links studies to employee
);

INSERT INTO studies VALUES
(1, 1, 'BA', 'State University', 2012, 'Human Resources'),
(2, 2, 'MSc', 'Tech Institute', 2017, 'Software Engineering'),
(3, 3, 'PhD', 'Ivy College', 2019, 'Marketing Research'),
(4, 4, 'MBA', 'Business School', 2018, 'Business Administration');

"""} ]

In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
-- Q1: Retrieve all employees and their base salary.
-- This query retrieves the names of all employees along with their base salary
-- by joining the 'employees' and 'salary' tables using 'ID_usr'.
-- A:
SELECT employees.name, salary.base
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Q2: Find employees who have an MBA degree.
-- This query finds the employees who hold an MBA degree by joining the
-- 'employees' and 'studies' tables, filtering by the degree 'MBA'.
-- A:
SELECT employees.name
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.degree = 'MBA';

-- Q3: Show the employees' total compensation (base salary + bonus).
-- This query calculates the total compensation for each employee by adding
-- the base salary and bonus, and displays the employee names and total compensation.
-- A:
SELECT employees.name, (salary.base + salary.bonus) AS total_compensation
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Q4: Find all employees who joined after 2018 and have a bonus greater than 1000.
-- This query retrieves the names of employees who joined after January 1, 2018
-- and have a bonus greater than 1000 by filtering the 'employees' and 'salary' tables.
-- A:
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE employees.hire_date > '2018-01-01' AND salary.bonus > 1000;

-- Q5: List all employees and their major subjects from the studies table.
-- This query retrieves the names of employees along with their major field of study
-- by joining the 'employees' and 'studies' tables.
-- A:
SELECT employees.name, studies.major
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;

"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

# Q1: Retrieve all employees and their base salary.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("Retrieve all employees and their base salary.", context_user))

```sql
SELECT employees.name, salary.base
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;
```


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Retrieve all employees and their base salary.", old_context_user))

This is your SQL:
```sql
SELECT employees.name, salary.salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;
```

This SQL query retrieves the names of all employees along with their base salary by joining the "employees" table with the "salary" table on the common column "ID_usr".


# Q2: Find employees who have an MBA degree.

In [8]:
#new
print(return_CCRMSQL("Find employees who have an MBA degree.", context_user))

```sql
SELECT employees.name
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.degree = 'MBA';
```


In [9]:
#old
print(return_CCRMSQL("Find employees who have an MBA degree.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 'MBA';
```

This SQL query selects the names of employees who have an MBA degree by joining the employees table with the studies table on the ID_usr field and filtering the results to only include rows where the educational level is 'MBA'.


# Q3: Show the employees' total compensation (base salary + bonus).

In [10]:
#new
context_user = context.copy()
print(return_CCRMSQL("Show the employees' total compensation (base salary + bonus).", context_user))

Here is the SQL query to show the employees' total compensation (base salary + bonus):

```sql
SELECT employees.name, (salary.base + salary.bonus) AS total_compensation
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;
```

This query will display the names of employees along with their total compensation calculated by adding their base salary and bonus.


In [11]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Show the employees' total compensation (base salary + bonus).", old_context_user))

This is your SQL:
```sql
SELECT e.ID_usr, (s.salary + s.bonus) AS total_compensation
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;
```

This SQL query retrieves the total compensation of employees by summing their base salary and bonus from the "salary" table, joining it with the "employees" table on the employee ID.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?